In [62]:
import os
import re
import time

import tweepy as tw
import pandas as pd
import numpy as np
from textblob import TextBlob

In [63]:
# Connection credentials
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'
# End connection credentials

# Define the AUTHHandler
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)
#End AUTHHandler

In [64]:
start_time = time.time()

# Define date and search Keyword
search_words = "#Covid"
date_since =pd.to_datetime('today').strftime("%Y-%m-%d")
#Define the cursor
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(50)
# Clean text
text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
# Create DataFrame 
users_locs = [[tweet.user.screen_name,tweet.user.name,tweet.user.verified,
         tweet.user.followers_count,tweet.user.friends_count,tweet.user.listed_count,
         tweet.retweet_count,tweet.favorite_count,tweet.retweeted,tweet.entities,
         tweet.user.favourites_count,
         tweet.user.location,tweet.created_at,tweet.text,
         re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text))),
         TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[0],
         TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[1]
              ] for tweet in tweets]
cols=columns=['screen_name','name','user_verification','followers_count','friends_count',
              'listed_count','retweet_count','favorite_count','retweeted','entities','favourites_count',
              'location','created_at','text','clean_text','sentiment_polarity','sentiment_subjectivity']
tweet_df = pd.DataFrame(data=users_locs, columns=cols)
tweet_df["sentiment_polarity_color"] = np.where(tweet_df["sentiment_polarity"]<0, 'red', 'green')
print("--- %s seconds ---" % (time.time() - start_time))
tweet_df.head()
# np.array_split(tweet_df, 3)

--- 2.792800188064575 seconds ---


,screen_name,name,user_verification,followers_count,friends_count,listed_count,retweet_count,favorite_count,retweeted,entities,favourites_count,location,created_at,text,clean_text,sentiment_polarity,sentiment_subjectivity,sentiment_polarity_color


In [25]:
# users_locs = [[tweet.user.screen_name,tweet.user.name,tweet.user.verified,
#          tweet.user.followers_count,tweet.user.friends_count,tweet.user.listed_count,
#          tweet.retweet_count,tweet.favorite_count,tweet.retweeted,tweet.entities,
#          tweet.user.favourites_count,
#          tweet.user.location,tweet.created_at,tweet.text,
#          re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text))),
#          TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[0],
#          TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[1]
#               ] for tweet in tweets]

In [20]:
# cols=columns=['screen_name','name','user_verification','followers_count','friends_count',
#               'listed_count','retweet_count','favorite_count','retweeted','entities','favourites_count',
#               'location','created_at','text','clean_text','sentiment_polarity','sentiment_subjectivity']

In [21]:
tweet_df = pd.DataFrame(data=users_locs, columns=cols)

In [22]:
tweet_df["sentiment_polarity_color"] = np.where(tweet_df["sentiment_polarity"]<0, 'red', 'green')

In [23]:
tweet_df.head()

,screen_name,name,user_verification,followers_count,friends_count,listed_count,retweet_count,favorite_count,retweeted,entities,favourites_count,location,created_at,text,clean_text,sentiment_polarity,sentiment_subjectivity,sentiment_polarity_color
0,yfir90,Firas,False,565,478,7,5,0,False,"{'hashtags': [{'text': 'COVID', 'indices': [45...",12303,He/Him/His,2021-03-19 17:12:57,RT @Drelkak: GREAT NEWS First Baby Born With #...,GREAT NEWS First Baby Born With COVID Antibod...,0.525000,0.541667,green
1,Mani_Cvs,Mani Kumar,False,9,78,0,2,0,False,"{'hashtags': [{'text': 'Rajasthan', 'indices':...",17,,2021-03-19 17:12:53,RT @ETNOWlive: #Rajasthan: #section144 CrPC im...,Rajasthan: section144 CrPC imposed in the sta...,0.000000,0.000000,green
2,UTRocketNursing,University of Toledo College of Nursing,False,408,596,5,0,0,False,"{'hashtags': [{'text': 'UToledo', 'indices': [...",945,"Toledo, OH",2021-03-19 17:12:52,We have done a lot at #UToledo to make sure ou...,We have done a lot at UToledo to make sure our...,0.166667,0.729630,green
3,homer2288,Jase Huang,False,26,100,0,290,0,False,"{'hashtags': [{'text': 'US', 'indices': [25, 2...",138,Home,2021-03-19 17:12:49,RT @SpokespersonCHN: The #US values &amp; rule...,The US values &amp; rules are not internation...,-0.350000,0.500000,red
4,rjsaxbi,Rod. Saxby,False,21,95,0,24,0,False,"{'hashtags': [{'text': 'Biden', 'indices': [12...",1307,,2021-03-19 17:12:48,RT @Ruptly: #Biden mistakenly refers to VP as ...,Biden mistakenly refers to VP as 'President H...,0.000000,0.000000,green
